In [ ]:
# Пример заполнения таблицы БД Tarantool

In [ ]:
## Вариант использования библиотеки tarantool

In [ ]:
import tarantool
import os

In [ ]:
# подключение к БД Tarantool
HOST=os.environ['TARANTOOL_HOST']
PORT=os.environ['TARANTOOL_PORT']
LOGIN=os.environ['TARANTOOL_LOGIN']
PASSWORD=os.environ['TARANTOOL_PASSWORD']

connection = tarantool.connect(host = HOST, port=int(PORT), user=LOGIN, password=PASSWORD)

In [ ]:
# имя таблицы 
tableName = 'table1'

# Пример 1

In [ ]:
values= [
        (1,1.0,'1'),
        (2,2.0,'2'),
        (3,3.0,'3')
    ]

In [ ]:
#имя таблицы всегда в верхнем регистре
tableName = tableName.upper()
#sys.stdout = open(os.devnull, 'w') #отключение stdout
for val in values:
    connection.insert(tableName, val)
#sys.stdout = sys.__stdout__ #включение stdout

# Пример 2

In [ ]:
values= [
        (4,4.0,'4'),
        (5,5.0,'5'),
        (6,6.0,'6')
    ]

In [ ]:
insertSQL = f'''insert into {tableName} 
    (
        c1, 
        c2,
        c3
    )
values 
    (
        :c1,
        :c2,
        :c3
    )
'''
for val in values:
    connection.execute(insertSQL, {'c1': val[0], 'c2': val[1], 'c3': val[2]})

# Пример 3

In [ ]:
import pandas

values = {'c1': [7, 8, 9], 'c2': [7.0, 8.0, 9.0], 'c3':['7', '8', '9']}
df = pandas.DataFrame( values)
data = df.to_dict(orient='index')
for key in data:
    print(data[key])
    connection.execute(insertSQL, data[key])

In [ ]:
connection.close()

## Заполенение средствами spark

Используется на больших объемах данных.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, LongType, StringType, IntegerType, DateType, TimestampType, FloatType
from pyspark.sql.functions import col, cast, date_trunc, sum, dayofweek, hour, dayofmonth, lit


In [ ]:
# запуск spark
spark = SparkSession.builder.appName('fill').getOrCreate()

In [ ]:
# подключение к БД Tarantool
HOST=os.environ['TARANTOOL_HOST']
PORT=os.environ['TARANTOOL_PORT']
LOGIN=os.environ['TARANTOOL_LOGIN']
PASSWORD=os.environ['TARANTOOL_PASSWORD']

jdbcURLTarantool = f'jdbc:tarantool://{HOST}:{PORT}?user={LOGIN}&password={PASSWORD}'

In [ ]:
# формирование dataframe
values= [
        (10,10.0,'10'),
        (11,11.0,'11'),
        (12,12.0,'12')
    ]

schemaData = StructType([ \
    StructField("c1",IntegerType(),True), 
    StructField("c2",FloatType(),True), 
    StructField("c3",StringType(),True)
  ])

df = spark.createDataFrame(values,schema=schemaData)

In [ ]:
df.write.format("jdbc").mode("append") \
              .option('url', jdbcURLTarantool) \
              .option('driver', 'org.tarantool.jdbc.SQLDriver') \
              .option("dbtable",tableName).save()

In [ ]:
spark.stop()